In [1]:
import tensorflow as tf

In [2]:
input_pattern = r"D:\MSCOCODATA\output\train-00???-of-00256"

In [3]:
data_files = []

In [4]:
data_files.extend(tf.gfile.Glob(input_pattern))

In [32]:
data_files

['D:\\MSCOCODATA\\output\\train-00000-of-00256',
 'D:\\MSCOCODATA\\output\\train-00001-of-00256',
 'D:\\MSCOCODATA\\output\\train-00002-of-00256',
 'D:\\MSCOCODATA\\output\\train-00003-of-00256',
 'D:\\MSCOCODATA\\output\\train-00004-of-00256',
 'D:\\MSCOCODATA\\output\\train-00005-of-00256',
 'D:\\MSCOCODATA\\output\\train-00006-of-00256',
 'D:\\MSCOCODATA\\output\\train-00007-of-00256',
 'D:\\MSCOCODATA\\output\\train-00008-of-00256',
 'D:\\MSCOCODATA\\output\\train-00009-of-00256',
 'D:\\MSCOCODATA\\output\\train-00010-of-00256',
 'D:\\MSCOCODATA\\output\\train-00011-of-00256',
 'D:\\MSCOCODATA\\output\\train-00012-of-00256',
 'D:\\MSCOCODATA\\output\\train-00013-of-00256',
 'D:\\MSCOCODATA\\output\\train-00014-of-00256',
 'D:\\MSCOCODATA\\output\\train-00015-of-00256',
 'D:\\MSCOCODATA\\output\\train-00016-of-00256',
 'D:\\MSCOCODATA\\output\\train-00017-of-00256',
 'D:\\MSCOCODATA\\output\\train-00018-of-00256',
 'D:\\MSCOCODATA\\output\\train-00019-of-00256',
 'D:\\MSCOCODATA\\ou

In [5]:
if not data_files:
    tf.logging.fatal("Found no input files matching %s", input_pattern)
else:
    tf.logging.info("Prefetching values from %d files matching %s",
                len(data_files), input_pattern)

INFO:tensorflow:Prefetching values from 256 files matching D:\MSCOCODATA\output\train-00???-of-00256


In [6]:
tfreader =tf.TFRecordReader()

In [7]:
filename_queue = tf.train.string_input_producer(data_files, shuffle=True, capacity=16, name="filename_queue")

In [8]:
values_queue = tf.RandomShuffleQueue(
    capacity=40000,
    min_after_dequeue=36800,
    dtypes=[tf.string],
    name="random_input_queue")

In [9]:
_, value = tfreader.read(filename_queue)

In [10]:
enqueue_ops = []

In [11]:
enqueue_ops.append(values_queue.enqueue([value]))

In [12]:
tf.train.queue_runner.add_queue_runner(tf.train.queue_runner.QueueRunner(values_queue, enqueue_ops))

In [13]:
enqueue_ops

[<tf.Operation 'random_input_queue_enqueue' type=QueueEnqueueV2>]

In [14]:
serialized_sequence_example = values_queue.dequeue()

In [15]:
serialized_sequence_example

<tf.Tensor 'random_input_queue_Dequeue:0' shape=<unknown> dtype=string>

In [16]:
image_feature_name = "image/data"
caption_feature_name = "image/caption_ids"

In [17]:
serialized_sequence_example

<tf.Tensor 'random_input_queue_Dequeue:0' shape=<unknown> dtype=string>

In [18]:
context, sequence = tf.parse_single_sequence_example(
    serialized_sequence_example,
    context_features={
      image_feature_name: tf.FixedLenFeature([], dtype=tf.string)
    },
    sequence_features={
      caption_feature_name: tf.FixedLenSequenceFeature([], dtype=tf.int64),
    })

In [19]:
encoded_image = context[image_feature_name]
caption = sequence[caption_feature_name]

In [20]:
encoded_image

<tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:0' shape=() dtype=string>

In [21]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [33]:
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    ima_vals = []
    cap_vals = []
    for i in range(5):        
        ima_val,cap_val = sess.run([encoded_image,caption])
        ima_vals.append(ima_val)
        cap_vals.append(cap_val)
    coord.request_stop()
    coord.join(threads)

In [37]:
len(ima_vals)

5

In [38]:
cap_vals

[array([  2,   0,  11,   9,   0, 312, 529,   4, 599,   4,   0, 174,   3,
          1], dtype=int64),
 array([  2,   0,  80,   9,   0,  11,  56,   8, 280,   4,   6,  75,   3,
          1], dtype=int64),
 array([  2,   0, 394,  58,   8,   0,  22,  82,   7,  28,   3,   1],
       dtype=int64),
 array([   2,    0, 1013,    5,  245,  654,    0, 1574,  168,    5,    0,
        2127,    1], dtype=int64),
 array([  2,   0,  34,   5,  19,  17, 120,   5,   0,  65,   4,   0,  24,
          3,   1], dtype=int64)]

In [39]:
ima_vals[2]

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x05XICC_PROFILE\x00\x01\x01\x00\x00\x05Happl\x02 \x00\x00scnrRGB XYZ \x07\xd3\x00\x07\x00\x01\x00\x00\x00\x00\x00\x00acspAPPL\x00\x00\x00\x00appl\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-appl\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0brXYZ\x00\x00\x01\x08\x00\x00\x00\x14gXYZ\x00\x00\x01\x1c\x00\x00\x00\x14bXYZ\x00\x00\x010\x00\x00\x00\x14wtpt\x00\x00\x01D\x00\x00\x00\x14chad\x00\x00\x01X\x00\x00\x00,rTRC\x00\x00\x01\x84\x00\x00\x00\x0egTRC\x00\x00\x01\x84\x00\x00\x00\x0ebTRC\x00\x00\x01\x84\x00\x00\x00\x0edesc\x00\x00\x04\xd8\x00\x00\x00ncprt\x00\x00\x04\x94\x00\x00\x00Adscm\x00\x00\x01\x94\x00\x00\x02\xfeXYZ \x00\x00\x00\x00\x00\x00tK\x00\x00>\x1d\x00\x00\x03\xcbXYZ \x00\x00\x00\x00\x00\x00Zs\x00\x00\x

In [40]:
len(ima_vals[2])

45425

In [41]:
len(ima_vals[3])

64994

In [42]:
len(ima_vals[4])

200468

In [45]:
len(ima_vals[1])

204401

In [46]:
len(ima_vals[0])

204336